In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bhe_N2qxhBpchzuAAU80DJl9JBS-AFpHWyojsSRYYYQT',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'waterqualityprediction-donotdelete-pr-vxlkfg7vfkorfd'
object_key = 'water_potability.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


Importing the Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

Data Pre-processing

In [3]:
dup = df.duplicated().sum()
print('Any Duplicate Value:',dup)

Any Duplicate Value: 0


In [4]:
df.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [5]:
df["ph"].fillna(value = df["ph"].mean(), inplace = True)
df["Sulfate"].fillna(value = df["Sulfate"].mean(), inplace = True)
df["Trihalomethanes"].fillna(value = df["Trihalomethanes"].mean(), inplace = True)

In [6]:
df.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [7]:
x = df.drop(['Potability'],axis=True)
y = df['Potability']

Splitting Data to Train and Test

In [8]:
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.32, random_state = 50)

In [9]:
x.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786
std,1.469956,32.879761,8768.570828,1.583085,36.142612,80.824064,3.308162,15.769881,0.780382
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000
25%,6.277673,176.850538,15666.690297,6.127421,317.094638,365.734414,12.065801,56.647656,3.439711
50%,7.080795,196.967627,20927.833607,7.130299,333.775777,421.884968,14.218338,66.396293,3.955028
75%,7.870050,216.667456,27332.762127,8.114887,350.385756,481.792304,16.557652,76.666609,4.500320
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000


In [10]:
y.describe()

count    3276.000000
mean        0.390110
std         0.487849
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: Potability, dtype: float64

Random Forest Classifier

In [11]:
RFC = RandomForestClassifier()
RFC.fit(x_train,y_train)
y_RFC = RFC.predict(x_test)

In [12]:
Acc_rfc= accuracy_score(y_RFC,y_test)
print( Acc_rfc)

0.6568160152526216


In [13]:
print(classification_report(y_RFC,y_test))

              precision    recall  f1-score   support

           0       0.89      0.66      0.76       861
           1       0.29      0.64      0.40       188

    accuracy                           0.66      1049
   macro avg       0.59      0.65      0.58      1049
weighted avg       0.79      0.66      0.70      1049



In [14]:
confusion_matrix(y_test,y_RFC)

array([[569,  68],
       [292, 120]])

AutoML Classifier

In [ ]:
import autosklearn.classification as classifier
# ac = classifier.AutoSklearnClassifier()
ac = classifier.AutoSklearnClassifier(time_left_for_this_task=520,per_run_time_limit=40)

In [ ]:
ac.fit(x_train,y_train)

AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      per_run_time_limit=40, time_left_for_this_task=520)

In [ ]:
ac_pred = ac.predict(x_test)

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
metrics.accuracy_score(y_test, ac_pred)

0.6549094375595805

AutoML with Hyper Parameters

In [15]:
model_params = {
    "Random Forest":
    {
        "model":RandomForestClassifier(),
        "params":
        {
            "n_estimators":[10, 50, 100],
            "max_features":["auto","sqrt","log2"],
            "max_depth":list(range(1,21,3))
        }
    }
    
}

In [16]:
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold, train_test_split
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats = 2)
scores = []
for model_name, params in model_params.items():
    rs = RandomizedSearchCV(params["model"], params["params"], cv = cv, n_iter = 10)
    rs.fit(x_train,y_train)
    scores.append([model_name, dict(rs.best_params_),rs.best_score_])

In [17]:
scores

[['Random Forest',
  {'n_estimators': 50, 'max_features': 'auto', 'max_depth': 13},
  0.6672620547185972]]

##IBM Deployment

In [18]:
!pip install -U ibm-watson-machine-learning

In [19]:
from ibm_watson_machine_learning import APIClient
import json

##Authenticate and Set Space

In [20]:
wml_credentials = {
    "apikey":"YeR_HFrqUOtVOM0cIcdQwFyHpjtDwehEpQQE0KG370_b",
    "url":"https://us-south.ml.cloud.ibm.com"
}

In [21]:
wml_client = APIClient(wml_credentials)

In [23]:
wml_client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------  ------------------------
ID                                    NAME      CREATED
ac36eaf2-d8a7-46e0-b8fa-86defb72e358  water-B7  2022-11-19T09:53:13.686Z
------------------------------------  --------  ------------------------


In [24]:
SPACE_ID= "ac36eaf2-d8a7-46e0-b8fa-86defb72e358"

In [25]:
wml_client.set.default_space(SPACE_ID)

'SUCCESS'

In [26]:
wml_client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt

##Save and Deploy Model

In [27]:
import sklearn
sklearn.__version__

'1.0.2'

In [29]:
MODEL_NAME = 'water-B7'
DEPLOYMENT_NAME = 'water-B7'
DEMO_MODEL = rs

In [30]:
# Set Python Version
software_spec_uid = wml_client.software_specifications.get_id_by_name('runtime-22.1-py3.9')

In [31]:
# Setup model meta
model_props = {
    wml_client.repository.ModelMetaNames.NAME: MODEL_NAME, 
    wml_client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0', 
    wml_client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid 
}

In [32]:
#Save model

model_details = wml_client.repository.store_model(
    model=DEMO_MODEL, 
    meta_props=model_props, 
    training_data=x_train, 
    training_target=y_train
)

In [33]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'label_column': 'Potability',
  'schemas': {'input': [{'fields': [{'name': 'ph', 'type': 'float64'},
      {'name': 'Hardness', 'type': 'float64'},
      {'name': 'Solids', 'type': 'float64'},
      {'name': 'Chloramines', 'type': 'float64'},
      {'name': 'Sulfate', 'type': 'float64'},
      {'name': 'Conductivity', 'type': 'float64'},
      {'name': 'Organic_carbon', 'type': 'float64'},
      {'name': 'Trihalomethanes', 'type': 'float64'},
      {'name': 'Turbidity', 'type': 'float64'}],
     'id': '1',
     'type': 'struct'}],
   'output': []},
  'software_spec': {'id': '12b83a17-24d8-5082-900f-0ab31fbfd3cb',
   'name': 'runtime-22.1-py3.9'},
  'type': 'scikit-learn_1.0'},
 'metadata': {'created_at': '2022-11-19T10:01:01.379Z',
  'id': '984e8e41-4fbb-4ca9-a4d3-2684c22f0a39',
  'modified_at': '2022-11-19T10:01:05.244Z',
  'name': 'water-B7',
  'owner': 'IBMid-6670007RXH',
  'resource_key': '7a118710-eeb1-4bfd-bb3d-633d26607d79',
  

In [34]:
model_id = wml_client.repository.get_model_id(model_details)
model_id

'984e8e41-4fbb-4ca9-a4d3-2684c22f0a39'

In [35]:
# Set meta
deployment_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME:DEPLOYMENT_NAME, 
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
}

In [36]:
# Deploy
deployment = wml_client.deployments.create(
    artifact_uid=model_id, 
    meta_props=deployment_props 
)



#######################################################################################

Synchronous deployment creation for uid: '984e8e41-4fbb-4ca9-a4d3-2684c22f0a39' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='16f81779-1bbf-45fa-8e34-3f5da54a7a43'
------------------------------------------------------------------------------------------------




In [ ]:
#https://us-south.ml.cloud.ibm.com/ml/v4/deployments/16f81779-1bbf-45fa-8e34-3f5da54a7a43/predictions?version=2022-11-19

In [ ]:
#3.45